### Tersor....Board....ing 

#### Watching the [introduction video](https://www.youtube.com/watch?v=eBbEDRsCmv4&t=1105s) [Demo code](https://github.com/decentralion/tf-dev-summit-tensorboard-tutorial)

If (Tensorboard == useful): 
    Install tensorflow https://www.tensorflow.org/install   
else 
    Close this notebook

#### start the tensorboard : 

> /Prometheus/src$ tensorboard --logdir=./logs_ann/
> 
> TensorBoard 1.8.0 at __http://*****:6006__ (Press CTRL+C to quit)
> 

Tips: if you open the 6006 url link but find nothing, calm down, double check the path. Wrong folder will give you empty board 


####  Wrong code could too, so the code begins here

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from os import listdir
from tensorflow.python.client import timeline

import calendar
import time
import arrow
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU,Input
from tensorflow.python.keras.optimizers import *
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.losses import mean_squared_error

## import data and tailor it for use

In [ ]:
import pickle
with open("data", "rb") as f:
    df_X = pickle.load(f)
    df_Y = pickle.load(f)

shift_iter = 24
X = df_X.values[shift_iter + 1:-shift_iter - 1]
Y = df_Y.values[shift_iter + 1:-shift_iter - 1]
# print(X.shape)
# print(Y.shape)
# print(list(df_X),list(df_Y))

ratio = 0.75  # for spliting train - test set
num_train = round(X.shape[0] * ratio)
x_train = X[:num_train]
y_train = Y[:num_train]
x_test = X[num_train:]
y_test = Y[num_train:]

x_scaler = MinMaxScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

num_x_signals = X.shape[1]
num_y_signals = Y.shape[1]

def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    You need to prepare the data before running this x_train_scaled , y_train_scaled
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)

            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx + sequence_length]
            y_batch[i] = y_train_scaled[idx:idx + sequence_length]

        yield (x_batch, y_batch)

### create a model from tensorflow using API : Keras 

In [ ]:
class TensorBoard_ValidateScalar(TensorBoard):
    def __init__(self, log_dir='./logs_for_tensorboard', **kwargs):
        
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super().__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super().set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary(value=[tf.Summary.Value(tag=name, simple_value=value)])
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        train_logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super().on_epoch_end(epoch, train_logs) # default send into scalars 
        
    def on_train_end(self, logs=None):
        super().on_train_end(logs)
        self.val_writer.close()

In [ ]:
class TensorBoard_ValidateHistogram(TensorBoard):
    def __init__(self, log_dir='./logs_for_tensorboard', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        self.training_log_dir = os.path.join(log_dir, 'training')
        super().__init__(self.training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        self.train_writer = tf.summary.FileWriter(self.training_log_dir)
        super().set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        print("\n",logs,"\n")
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            
            # histogram in Tensorboard
            values = np.array(value)
            # Create histogram using numpy
            counts, bin_edges = np.histogram(values, bins=1000)
            # Fill fields of histogram proto
            hist = tf.HistogramProto()
            hist.min = float(np.min(values))
            hist.max = float(np.max(values))
            hist.num = int(np.prod(values.shape))
            hist.sum = float(np.sum(values))
            hist.sum_squares = float(np.sum(values ** 2))
            bin_edges = bin_edges[1:]

            # Add bin edges and counts
            for edge in bin_edges:
                hist.bucket_limit.append(edge)
            for c in counts:
                hist.bucket.append(c)
            summary = tf.Summary(value=[tf.Summary.Value(tag=name, histo=hist)])
            
            self.val_writer.add_summary(summary, epoch)

        self.val_writer.flush()

        # Pass the train logs to `TensorBoard.on_epoch_end`
        train_logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        for name, value in train_logs.items():
            
            # histogram in Tensorboard
            values = np.array(value)
            # Create histogram using numpy
            counts, bin_edges = np.histogram(values, bins=1000)
            # Fill fields of histogram proto
            hist = tf.HistogramProto()
            hist.min = float(np.min(values))
            hist.max = float(np.max(values))
            hist.num = int(np.prod(values.shape))
            hist.sum = float(np.sum(values))
            hist.sum_squares = float(np.sum(values ** 2))
            bin_edges = bin_edges[1:]

            # Add bin edges and counts
            for edge in bin_edges:
                hist.bucket_limit.append(edge)
            for c in counts:
                hist.bucket.append(c)
            summary = tf.Summary(value=[tf.Summary.Value(tag=name, histo=hist)])
            self.train_writer.add_summary(summary, epoch)
        self.train_writer.flush()
        
        # write the gradient and all the metrics values into tensorboard 
        super().on_epoch_end(epoch, logs) 

    def on_train_end(self, logs=None):
        super().on_train_end(logs)
        self.val_writer.close()
        self.train_writer.close()

In [ ]:
model = Sequential()

model.add(Dense(num_x_signals ,
              activation='hard_sigmoid',
              name = "hidden",
              input_shape=(None, num_x_signals,)))

model.add(Dense(num_y_signals, 
                activation='linear',
                name = "output"))

model.compile(loss = mean_squared_error,
          optimizer = Adam(lr=1e-2),
          metrics = ['accuracy','mae','mean_squared_error','mape'], # this part will show up in tensorboard
          options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE), 
          run_metadata= tf.RunMetadata(), # these two show up in Chrome Trace
             )

model.summary()

In [ ]:
%%time

log_dir="./logs_tensorboard_demo/{}".format(arrow.now().format('YYYY-MM-DD-HH-MM-SS'))
print(log_dir)

model.fit_generator(generator = batch_generator(batch_size=200, sequence_length=168) ,
                    epochs=10,
                    verbose = 0,
                    steps_per_epoch=80,
                    validation_data=validation_data,
                    callbacks=[
                           TensorBoard_ValidateHistogram(
                           log_dir,
                           histogram_freq = 1,
                           write_graph = True, # visualize the graph in TensorBoard
                           write_images = True, #  write model weights to visualize as image in TensorBoard
                           write_grads = True, # visualize training gradient histograms in TensorBoard
                                  )
                            ])

In [ ]:
import os
os._exit(00)

In [ ]:



import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np
import arrow

log_dir="./logs_tensorboard_demo/{}".format(arrow.now().format('YYYY-MM-DD-HH-MM-SS'))



# making up some random data
x_data = np.linspace(-1,1,300)[:,np.newaxis] # flatten your arrary into list
noise = np.random.normal(0,0.05,x_data.shape)
y_data = np.square(x_data)-0.5+noise

# tf.summary.image('input',tf.reshape(x_data,[1,300,1,1]))
with tf.name_scope('input_layer'):
    x = tf.placeholder(tf.float32,[None,1],name='x_input')
    y = tf.placeholder(tf.float32,[None,1],name='y_input')

    
with tf.name_scope('hidden_layer'):
    with tf.name_scope('W'):
        W1 = tf.Variable(tf.random_normal([1,10]))
        tf.summary.histogram('hidden-weight',W1)

    with tf.name_scope('B'):
        b1 = tf.constant(0.5,shape=[1,10])
        tf.summary.histogram('hidden-bias',b1)

    with tf.name_scope('Activation'):
        a = tf.matmul(x,W1)+b1
        tf.summary.histogram('hidden-pre-active',a)

    hidden = tf.nn.softmax(a)
    tf.summary.histogram('hidden-activation',hidden)

with tf.name_scope('output_layer'):
    with tf.name_scope('W'):
        W2 = tf.Variable(tf.random_normal([10,1]))
        tf.summary.histogram('output-weight', W2)

    with tf.name_scope('B'):
        b2 = tf.constant(0.1, shape=[1, 1])
        tf.summary.histogram('output-bias', b2)

    with tf.name_scope('Activation'):
        a = tf.matmul(hidden, W2) + b2
        tf.summary.histogram('output-pre-active', a)

    output = (a)
    tf.summary.histogram('output-activation', output)


with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(y-output),reduction_indices=[1]))
    tf.summary.scalar('loss',loss)


with tf.name_scope('train'):
    train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

init = tf.global_variables_initializer()
merge = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter(log_dir,sess.graph)

    for i in range(500) :
        sess.run(train,feed_dict={x:x_data,y:y_data})
        result = sess.run(merge,feed_dict={x:x_data,y:y_data})
        writer.add_summary(result,i)

In [ ]:
from io import StringIO
class Logger(object):
    """Logging in tensorboard without tensorflow ops."""

    def __init__(self, log_dir):
        """Creates a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)

    def log_scalar(self, tag, value, step):
        """Log a scalar variable.
        Parameter
        ----------
        tag : basestring
            Name of the scalar
        value
        step : int
            training iteration
        """
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag,
                                                     simple_value=value)])
        self.writer.add_summary(summary, step)
        self.writer.flush()
    def log_images(self, tag, images, step):
        """Logs a list of images."""

        im_summaries = []
        for nr, img in enumerate(images):
            # Write the image to a string
            s = StringIO()
            plt.imsave(s, img, format='png')

            # Create an Image object
            img_sum = tf.Summary.Image(encoded_image_string=s.getvalue(),
                                       height=img.shape[0],
                                       width=img.shape[1])
            # Create a Summary value
            im_summaries.append(tf.Summary.Value(tag='%s/%d' % (tag, nr),
                                                 image=img_sum))

        # Create and write Summary
        summary = tf.Summary(value=im_summaries)
        self.writer.add_summary(summary, step)
        self.writer.flush()
    def log_histogram(self, tag, values, step, bins=1000):
        """Logs the histogram of a list/vector of values."""
        # Convert to a numpy array
        values = np.array(values)

        # Create histogram using numpy
        counts, bin_edges = np.histogram(values, bins=bins)

        # Fill fields of histogram proto
        hist = tf.HistogramProto()
        hist.min = float(np.min(values))
        hist.max = float(np.max(values))
        hist.num = int(np.prod(values.shape))
        hist.sum = float(np.sum(values))
        hist.sum_squares = float(np.sum(values ** 2))

        # Requires equal number as bins, where the first goes from -DBL_MAX to bin_edges[1]
        # See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/summary.proto#L30
        # Thus, we drop the start of the first bin
        bin_edges = bin_edges[1:]

        # Add bin edges and counts
        for edge in bin_edges:
            hist.bucket_limit.append(edge)
        for c in counts:
            hist.bucket.append(c)

        # Create and write Summary
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, histo=hist)])
        self.writer.add_summary(summary, step)
        self.writer.flush()